# Задание 1
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 2
* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

# Задание 3 *
* Напишете скрипт, который сможет ежедневно собирать данные по цене, количестве и объему строящихся объектов самостоятельно и ежедневно
* Попробуйте ускорить процесс
* Добавьте возможность добавления данных в БД с указанием текущей даты.

In [147]:
import requests
import json
import pandas as pd
import sqlite3
import re, os
from tqdm.auto import tqdm

## Список объектов 

In [150]:
limit_ = 1000
all_obj_ids = []
for offset_ in tqdm(range(0,12000,1000)):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    all_obj_ids.extend(objids)

  0%|          | 0/12 [00:00<?, ?it/s]

In [151]:
len(all_obj_ids)

10026

# Данные по определенному объекту

In [152]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/45434'

In [153]:
res = requests.get(url)

In [154]:
df = pd.DataFrame()
for i in tqdm(all_obj_ids):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{i}'
    res = requests.get(url)
    #df = pd.json_normalize(objects_data['data']['list'])
    df = pd.concat([df, pd.json_normalize(res.json())], axis=0)

  0%|          | 0/10026 [00:00<?, ?it/s]

In [155]:
df.head(10)

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.metro.colors,data.transportDistIndex,data.transportDistIndexValue,data.greenAreaIndexValue,data.airQualityIndexValue,data.complexShortNm,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm
0,0,13399,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,13398,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,13383,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,13397,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,13391,28101,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,23682,64486,6811,84 ВЫСОТА,84 ВЫСОТА,84 ВЫСОТА,0,1,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,36321,319464,10195,CЗ АЗИЯ-КОНТРАКТ,CЗ АЗИЯ-КОНТРАКТ,CПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК АЗИЯ-КОНТРАКТ,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,36912,337586,10296,CЗ СТРОЙРЕСУРС,CЗ СТРОЙРЕСУРС,CПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК СТРОЙРЕСУРС,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,44076,275899,9115,XXI ВЕК СТРОЙ,XXI ВЕК СТРОЙ,XXI ВЕК СТРОЙ,0,1,0,...,[#D80707],1 / 10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,23494,35375,4074,А-ЛЕКС,А-ЛЕКС,А-ЛЕКС,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
# Сохраняем в Excel
df.to_excel("df_excel.xlsx")  

In [157]:
# Сохраняем в pickle
df_pd.to_pickle("df_pickle.pkl")

In [158]:
# Cохраняем pandas dataframe в БД
conn = sqlite3.connect('test_database')
df_pd.to_sql('dom_rf', conn, if_exists='replace', index = False)
conn.commit()
conn.close()

In [159]:
# Проверка БД
conn = sqlite3.connect('test_database')
db_data = pd.read_sql('SELECT * FROM dom_rf', conn)
db_data.head(5)

,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13398,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,4812.98,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,None,None,NaN,None,None,None
1,13383,72,г Тюмень,4,20,68,2022-06-30,6289.98,1,0,...,Акционерное общество,АО,6.994210e+08,7701651356,None,None,NaN,None,None,None
2,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,None,None,NaN,None,None,None
3,13391,72,"г Тюмень, ул Профсоюзная",3,12,44,2022-06-30,4293.67,1,0,...,Акционерное общество,АО,6.994210e+08,7701651356,None,None,NaN,None,None,None
4,23682,47,д Агалатово,10,10,54,2022-06-30,2748.15,1,0,...,Общество с ограниченной ответственностью,ООО,5.562608e+09,7806536079,"Жилой комплекс ""84 высота""",None,NaN,None,None,None


Сравните среднюю цену квадратного метра в городах этого региона

In [160]:
vrn = 36

In [161]:
df_vrn_regs = df[df['data.region'] == vrn]

In [163]:
df_f = df_vrn_regs['data.address'].str.split(',', expand=True)
df_vrn_regs['city'] = df_f[0]
df_vrn_regs['city'] = df_f
df_vrn_regs_avg = df_vrn_regs.groupby(['city'])['data.objPriceAvg'].mean().sort_values(ascending=False)
df_2 = pd.DataFrame(df_vrn_regs_avg)
df_2

<ipython-input-163-bff1ace348e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vrn_regs['city'] = df_f[0]
<ipython-input-163-bff1ace348e8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vrn_regs['city'] = df_f


,data.objPriceAvg
city,
п Солнечный,80026.600000
х Ветряк,76537.000000
г Воронеж,67033.250000
с Семилуки,64316.500000
п Отрадное,62120.500000
с Ямное,61770.200000
п Шуберское,53958.333333
с Александровка,53907.500000
с Новая Усмань,50313.000000


Определите количество строящихся объектов и квартир в регионах

In [164]:
cnt_obj = df.groupby('data.region')['data.id'].count()
cnt_obj
df_2 = pd.DataFrame(cnt_obj)
df_2.rename(columns = {'data.id':'Кол-во объектов'}, inplace = True)
df_2

,Кол-во объектов
data.region,
1,45
2,245
3,43
4,5
5,20
...,...
78,679
86,61
89,34


In [165]:
cnt_rm = df.groupby('data.region')['data.objFlatCnt'].sum()
cnt_rm
df_3 = pd.DataFrame(cnt_rm)
df_3.rename(columns = {'data.id':'Кол-во квартир'}, inplace = True)
df_3

,data.objFlatCnt
data.region,
1,5019
2,58429
3,5810
4,356
5,3546
...,...
78,196577
86,8696
89,2557


Определите объем строящегося жилья в квадратных метрах

In [166]:
df_sm = df['data.objFlatSq'].astype(float).sum()
df_sm

95200011.11999999